In [133]:
from datetime import datetime, timedelta
import os
import shutil
import re
import sqlparse

In [134]:
def read_contents_file(file_path, encoding_scheme='cp1252'):
    inputfile = open(file_path, 'r', encoding=encoding_scheme)
    return inputfile.read()

In [135]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = file_data
        file.write(file_string)

In [136]:
input_path_folder = "Original"
input_base_folder_regex = re.compile("^" + input_path_folder, re.IGNORECASE)
output_path_folder = "Minimized"
keyword_list = ["cast(", "convert(", "varchar(", "as date", "as datetime", "coalesce(", "char(",
"trim(", "rtrim(", "ltrim(", "substring(", "substr(", "replace(","upper(","left(","len(","right("
"getdate(", "dateadd(", "datepart(", "todate("]

In [137]:
#get the DDLs for the tables from teradata 
def format_sql_code():
    dt1 = datetime.now()

    try:
        if os.path.exists(output_path_folder):
            shutil.rmtree(output_path_folder, ignore_errors=True)

        # for filename in os.listdir(input_path_folder):
        for folder_path, dirs, files in os.walk(input_path_folder):
            for filename in files:
                file_path = f"{folder_path}\{filename}"
                sqltext = read_contents_file(file_path)
                # Formatting SQL - Uppercase keywords, lowercasing database identifiers and stripping comments
                formattedtext = sqlparse.format(sqltext, reindent=False, keyword_case='upper', identifier_case='lower',strip_comments=True)
                # Remove whitespace and minimizing the code
                # formattedtext = re.sub(r"[[\]]", "", re.sub(" ,", ",", re.sub(" +", " ", re.sub(",", ", ", re.sub("[\r]|[\n]|[\t]"," ",formattedtext)))))
                formattedtext =re.sub(" ,", ",", re.sub(" +", " ", re.sub(",", ", ", re.sub("[\r]|[\n]|[\t]"," ",formattedtext))))
                # sqltext = re.sub(r"[[\]]", "", re.sub(" ,", ",", re.sub(" +", " ", re.sub(",", ", ", re.sub(r"\n"," ",sqltext)))))
                # Uppercase missed keywords
                for keyword in keyword_list:
                    keyword_regex = re.compile(re.escape(keyword), re.IGNORECASE)
                    formattedtext = re.sub(keyword_regex, keyword.upper(), formattedtext)
                new_folder_path = re.sub(input_base_folder_regex, output_path_folder, folder_path)
                if not os.path.exists(new_folder_path):
                    os.makedirs(new_folder_path)
                new_file_path = f"{new_folder_path}\{filename}"
                write_file_local(new_file_path, formattedtext)

    except Exception as e1:
        print(e1)
        print("File Name : {}".format(file_path))
        pass
    
    dt2 = datetime.now()
    print(dt2-dt1)

In [138]:
print("Begin of Processing")

format_sql_code()

print("End of Processing")

Begin of Processing
0:00:04.841517
End of Processing
